In [1]:
import osmnx as ox
import geopandas as gpd
import numpy as npm
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from collections import deque
import datetime

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
CRS_PLOT = 'EPSG:4326'
DATA_DIR = '../data'
EXPORTS_DIR = f'{DATA_DIR}/exports'

In [3]:
stops_df = pd.read_csv(f'{DATA_DIR}/gtfs_20210428/stops.txt')
stop_times_df = pd.read_csv(f'{DATA_DIR}/gtfs_20210428/stop_times.txt')
stop_times_df = stop_times_df.drop(columns=['departure_time', 'pickup_type', 'drop_off_type']).rename(columns={'arrival_time': 'time'})

In [4]:
stop_times_df[stop_times_df['trip_id'] == 'EN_B1-Weekday-076700_B25_220']

,trip_id,time,stop_id,stop_sequence
64137,EN_B1-Weekday-076700_B25_220,12:47:00,308626,1
64138,EN_B1-Weekday-076700_B25_220,12:48:13,308025,2
64139,EN_B1-Weekday-076700_B25_220,12:50:05,302355,3
64140,EN_B1-Weekday-076700_B25_220,12:50:46,306437,4
64141,EN_B1-Weekday-076700_B25_220,12:51:37,302356,5
64142,EN_B1-Weekday-076700_B25_220,12:52:37,302357,6
64143,EN_B1-Weekday-076700_B25_220,12:53:21,302359,7
64144,EN_B1-Weekday-076700_B25_220,12:54:20,302358,8
64145,EN_B1-Weekday-076700_B25_220,12:55:00,303011,9
64146,EN_B1-Weekday-076700_B25_220,12:59:58,307460,10


In [5]:
stop_times_idf = stop_times_df.set_index(['trip_id', 'stop_sequence'])

In [6]:
stop_times_idf

time  stop_id
trip_id                          stop_sequence                   
EN_B1-Weekday-028500_SBS82_901   1              04:45:00   304083
                                 2              04:47:37   304085
                                 3              04:49:08   308865
                                 4              04:51:39   306950
                                 5              04:53:54   304092
...                                                  ...      ...
UP_B1-Weekday-SDon-098100_B6_217 31             17:02:47   308648
                                 32             17:03:23   308050
                                 33             17:03:54   300421
                                 34             17:04:40   300422
                                 35             17:05:00   901004

[2058923 rows x 2 columns]

In [7]:
starts_df = stop_times_df[stop_times_df['stop_sequence'] == 1].drop(columns=['stop_sequence'])

In [8]:
stop_times_idf.iloc[0]

time       04:45:00
stop_id      304083
Name: (EN_B1-Weekday-028500_SBS82_901, 1), dtype: object

In [9]:
starts_df

,trip_id,time,stop_id
0,EN_B1-Weekday-028500_SBS82_901,04:45:00,304083
23,EN_B1-Weekday-034800_SBS82_901,05:48:00,801192
46,EN_B1-Weekday-040300_SBS82_901,06:43:00,304083
69,EN_B1-Weekday-048000_SBS82_901,08:00:00,801192
92,EN_B1-Weekday-029900_SBS82_902,04:59:00,304083
...,...,...,...
2058616,UP_B1-Weekday-SDon-133600_B6_293,22:16:00,901022
2058684,UP_B1-Weekday-SDon-134800_B6_296,22:28:00,901022
2058752,UP_B1-Weekday-SDon-136000_B6_307,22:40:00,901022
2058820,UP_B1-Weekday-SDon-137200_B6_317,22:52:00,901022


In [49]:
b25_starts_df = stop_times_df[(stop_times_df['stop_id'] == 308626) & (stop_times_df['stop_sequence'] == 1)]
b25_ends_df = stop_times_df[(stop_times_df['stop_id'] == 901211) & (stop_times_df['stop_sequence'] == 42)]

b25_starts_df = b25_starts_df[b25_starts_df['time'].str[0:2].astype(int) < 24]
b25_end_df = b25_starts_df[b25_starts_df['time'].str[0:2].astype(int) < 24]

b25_trip_ids = pd.merge(b25_starts_df[['trip_id']], b25_ends_df[['trip_id']])
print(len(b25_trip_ids))

370


In [50]:
b25_stop_times_df = pd.merge(stop_times_df, b25_trip_ids)
b25_weekday_stop_times_df = b25_stop_times_df[b25_stop_times_df['trip_id'].str.contains('Weekday')]

In [51]:
b25_weekday_stop_times_idf = b25_stop_times_df.set_index(['trip_id', 'stop_sequence'])
b25_weekday_stop_times_idf

time  stop_id
trip_id                           stop_sequence                   
EN_B1-Weekday-005800_B25_201      1              00:58:00   308626
                                  2              00:58:55   308025
                                  3              01:00:19   302355
                                  4              01:00:49   306437
                                  5              01:01:28   302356
...                                                   ...      ...
EN_B1-Weekday-SDon-133200_B25_238 38             23:03:04   307259
                                  39             23:04:27   307260
                                  40             23:05:32   307631
                                  41             23:07:52   307263
                                  42             23:10:00   901211

[15540 rows x 2 columns]

In [52]:
len(b25_trip_ids[b25_trip_ids['trip_id'].str.contains('Weekday')])

208

In [72]:
b25_stops = list(stop_times_df[stop_times_df['trip_id'] == b25_starts_df.iloc[0]['trip_id']]['stop_id'])

In [77]:
df = b25_weekday_stop_times_df
df = df.pivot(index='trip_id', columns='stop_id', values='time')
df = df[b25_stops]
df = df.sort_values(by=[308626])
df = df.drop_duplicates(subset=[308626, 901211], keep="last")

In [78]:
df.to_csv(f'{EXPORTS_DIR}/b25_weekday_stop_times.csv')

stop_id,308626,308025,302355,306437,302356,302357,302359,302358,303011,307460,...,307253,307254,307255,307256,307257,307259,307260,307631,307263,901211
trip_id,,,,,,,,,,,,,,,,,,,,,
EN_B1-Weekday-SDon-005800_B25_201,00:58:00,00:58:55,01:00:19,01:00:49,01:01:28,01:02:13,01:02:46,01:03:30,01:04:00,01:06:42,...,01:27:53,01:29:11,01:30:00,01:31:13,01:32:26,01:33:42,01:34:58,01:35:57,01:38:04,01:40:00
EN_B1-Weekday-SDon-014400_B25_201,02:24:00,02:24:46,02:25:56,02:26:21,02:26:53,02:27:31,02:27:58,02:28:35,02:29:00,02:31:15,...,02:49:53,02:51:11,02:52:00,02:52:59,02:53:57,02:54:57,02:55:58,02:56:45,02:58:27,03:00:00
EN_B1-Weekday-SDon-021800_B25_201,03:38:00,03:38:46,03:39:56,03:40:21,03:40:53,03:41:31,03:41:58,03:42:35,03:43:00,03:45:15,...,04:03:53,04:05:11,04:06:00,04:06:59,04:07:57,04:08:57,04:09:58,04:10:45,04:12:27,04:14:00
EN_B1-Weekday-SDon-029200_B25_201,04:52:00,04:52:46,04:53:56,04:54:21,04:54:53,04:55:31,04:55:58,04:56:35,04:57:00,04:59:15,...,05:17:53,05:19:11,05:20:00,05:20:59,05:21:57,05:22:57,05:23:58,05:24:45,05:26:27,05:28:00
EN_B1-Weekday-SDon-033200_Q56_453,05:32:00,05:33:04,05:34:42,05:35:17,05:36:02,05:36:55,05:37:34,05:38:25,05:39:00,05:41:42,...,06:03:37,06:05:05,06:06:00,06:07:13,06:08:26,06:09:42,06:10:58,06:11:57,06:14:04,06:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EN_B1-Weekday-130900_B83_317,21:49:00,21:50:23,21:52:28,21:53:14,21:54:11,21:55:19,21:56:09,21:57:15,21:58:00,22:01:37,...,22:32:49,22:34:46,22:36:00,22:37:21,22:38:41,22:40:04,22:41:27,22:42:32,22:44:52,22:47:00
EN_B1-Weekday-133200_B25_238,22:12:00,22:13:23,22:15:28,22:16:14,22:17:11,22:18:19,22:19:09,22:20:15,22:21:00,22:24:37,...,22:55:49,22:57:46,22:59:00,23:00:21,23:01:41,23:03:04,23:04:27,23:05:32,23:07:52,23:10:00
EN_B1-Weekday-135700_B25_235,22:37:00,22:38:23,22:40:28,22:41:14,22:42:11,22:43:19,22:44:09,22:45:15,22:46:00,22:49:37,...,23:18:05,23:19:52,23:21:00,23:22:13,23:23:26,23:24:42,23:25:58,23:26:57,23:29:04,23:31:00


In [74]:
df

stop_id,302355,302356,302357,302358,302359,302368,302369,302370,302371,302372,...,307259,307260,307263,307460,307491,307631,307926,308025,308626,901211
trip_id,,,,,,,,,,,,,,,,,,,,,
EN_B1-Weekday-SDon-005800_B25_201,01:00:19,01:01:28,01:02:13,01:03:30,01:02:46,01:12:36,01:13:41,01:15:00,01:15:46,01:16:26,...,01:33:42,01:34:58,01:38:04,01:06:42,01:08:46,01:35:57,01:25:19,00:58:55,00:58:00,01:40:00
EN_B1-Weekday-SDon-014400_B25_201,02:25:56,02:26:53,02:27:31,02:28:35,02:27:58,02:36:05,02:36:57,02:38:00,02:38:39,02:39:14,...,02:54:57,02:55:58,02:58:27,02:31:15,02:32:58,02:56:45,02:47:19,02:24:46,02:24:00,03:00:00
EN_B1-Weekday-SDon-021800_B25_201,03:39:56,03:40:53,03:41:31,03:42:35,03:41:58,03:50:05,03:50:57,03:52:00,03:52:39,03:53:14,...,04:08:57,04:09:58,04:12:27,03:45:15,03:46:58,04:10:45,04:01:19,03:38:46,03:38:00,04:14:00
EN_B1-Weekday-SDon-029200_B25_201,04:53:56,04:54:53,04:55:31,04:56:35,04:55:58,05:04:05,05:04:57,05:06:00,05:06:39,05:07:14,...,05:22:57,05:23:58,05:26:27,04:59:15,05:00:58,05:24:45,05:15:19,04:52:46,04:52:00,05:28:00
EN_B1-Weekday-SDon-033200_Q56_453,05:34:42,05:36:02,05:36:55,05:38:25,05:37:34,05:47:36,05:48:41,05:50:00,05:50:46,05:51:26,...,06:09:42,06:10:58,06:14:04,05:41:42,05:43:46,06:11:57,06:00:44,05:33:04,05:32:00,06:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EN_B1-Weekday-130900_B83_317,21:52:28,21:54:11,21:55:19,21:57:15,21:56:09,22:09:39,22:11:10,22:13:00,22:14:12,22:15:15,...,22:40:04,22:41:27,22:44:52,22:01:37,22:04:21,22:42:32,22:28:59,21:50:23,21:49:00,22:47:00
EN_B1-Weekday-133200_B25_238,22:15:28,22:17:11,22:18:19,22:20:15,22:19:09,22:32:39,22:34:10,22:36:00,22:37:12,22:38:15,...,23:03:04,23:04:27,23:07:52,22:24:37,22:27:21,23:05:32,22:51:59,22:13:23,22:12:00,23:10:00
EN_B1-Weekday-135700_B25_235,22:40:28,22:42:11,22:43:19,22:45:15,22:44:09,22:57:39,22:59:10,23:01:00,23:01:59,23:02:51,...,23:24:42,23:25:58,23:29:04,22:49:37,22:52:21,23:26:57,23:14:34,22:38:23,22:37:00,23:31:00
